In [1]:
import os
from PIL import Image
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info
import torch
import gc

# Define paths and model configuration
model_id = "Qwen/Qwen2-VL-2B-Instruct"
cases_dir = '/media/RLAB-Disk01/(final)merged_images_with_labels_order_and_folders_mask_normalized/'
cache_dir = '/media/RLAB-Disk01/Large-Language-Models-Weights'
offload_folder = '/media/RLAB-Disk01/Large-Language-Models-Weights'

In [2]:
# os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:512'
# torch.cuda.set_per_process_memory_fraction(0.8)

In [3]:
# Collect list of cases that do not have 'qwen-vl-72b-response.txt'
cases_to_process = []
for case in os.listdir(cases_dir):
    case_dir = os.path.join(cases_dir, case)
    if os.path.isdir(case_dir):
        response_path = os.path.join(case_dir, 'qwen-vl-2b-response.txt')
        if not os.path.exists(response_path):
            cases_to_process.append(case)
print(f"Found {len(cases_to_process)} cases to process.")

Found 40 cases to process.


In [4]:
# Load the model and processor
print("Loading model and processor...")
model = Qwen2VLForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    cache_dir=cache_dir,
    offload_folder=offload_folder,
    attn_implementation="flash_attention_2"
)
processor = AutoProcessor.from_pretrained(model_id)
print("Model and processor loaded successfully!\n")


Loading model and processor...


`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Model and processor loaded successfully!



In [5]:
def load_image(image_path):
    try:
        img = Image.open(image_path)
        return img
    except Exception as e:
        print(f"Error loading image {image_path}: {e}")
        return None

In [6]:
print("Processing cases...--->")
print(cases_to_process)

Processing cases...--->
['RHUH-0019', 'RHUH-0001', 'RHUH-0002', 'RHUH-0003', 'RHUH-0004', 'RHUH-0005', 'RHUH-0006', 'RHUH-0007', 'RHUH-0008', 'RHUH-0009', 'RHUH-0010', 'RHUH-0011', 'RHUH-0012', 'RHUH-0013', 'RHUH-0014', 'RHUH-0015', 'RHUH-0016', 'RHUH-0017', 'RHUH-0018', 'RHUH-0020', 'RHUH-0021', 'RHUH-0022', 'RHUH-0023', 'RHUH-0024', 'RHUH-0025', 'RHUH-0026', 'RHUH-0027', 'RHUH-0028', 'RHUH-0029', 'RHUH-0030', 'RHUH-0031', 'RHUH-0032', 'RHUH-0033', 'RHUH-0034', 'RHUH-0035', 'RHUH-0036', 'RHUH-0037', 'RHUH-0038', 'RHUH-0039', 'RHUH-0040']


In [7]:
# Initialize list to track failed cases
failed_cases = []

# Iterate through cases in the directory
for case in cases_to_process:
    case_dir = os.path.join(cases_dir, case)
    if not os.path.isdir(case_dir):
        continue  # Skip non-directory files

    image_files = [f for f in os.listdir(case_dir) if f.lower().endswith('.png')]
    print(f"Found {len(image_files)} image files for case {case}")

    clinical_information_path = os.path.join(case_dir, 'diagnostic_prompt.txt')
    if not os.path.exists(clinical_information_path):
        print(f"Missing clinical information file for case: {case}")
        continue


    print(f"Reading clinical information for case {case}")    
    clinical_information = open(clinical_information_path, 'r', encoding='utf-8').read()

    system_prompt = """Consider that you are a professional radiologist with several years of experience and you are now treating a patient. Write a fully detailed diagnosis report for this case, avoiding any potential hallucination and paying close attention to all of the batch images attached to this message.

Use the following structure for the report:

## Radiologist's Report

### Patient Information:
- *Age:* 65
- *Sex:* Male
- *Days from earliest imaging to surgery:* 1
- *Histopathological Subtype:* Glioblastoma
- *WHO Grade:* 4
- *IDH Status:* Mutant
- *Preoperative KPS:* 80
- *Preoperative Contrast-Enhancing Tumor Volume (cm³):* 103.21
- *Preoperative T2/FLAIR Abnormality (cm³):* 36.29
- *Extent of Resection (EOR %):* 100.0
- *EOR Type:* Gross Total Resection (GTR)
- *Adjuvant Therapy:* Radiotherapy (RT) + Temozolomide (TMZ)
- *Progression-Free Survival (PFS) Days:* 649
- *Overall Survival (OS) Days:* 736

#### Tumor Characteristics:

#### Segmentation Analysis:

#### Surgical Considerations:

### Clinical Summary:

### Recommendations:

### Prognostic Considerations:

### Follow-Up Plan:

### Additional Notes*(if any)*:

Ensure all findings from all of the images and clinical data provided. Please mention at the end of the report how many images were reviewed."""

    user_prompt = f"""You will be given batches of images, which are different sequences of MRI scans. 
    The images are for patients who are likely to have a brain tumor. Each image will contain up to 10 slices for 5 different sequences and the segmentation masks for the tumor at the bottom row of the image. 
    Additional clinical data about the patient is: 
    {clinical_information}"""

    # Collect the last image
    last_image = None
    for image_file in image_files:
        image_path = os.path.join(case_dir, image_file)
        img = load_image(image_path)
        if img is not None:
            last_image = {"type": "image", "image": img}
            print(f"Loaded image: {image_file} for case {case}")
    if last_image is None:
        print(f"No valid images found for case: {case}")
        continue

    # Prepare messages for Qwen2-VL
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": [last_image, {"type": "text", "text": user_prompt}]},
    ]
    print(f"Messages prepared for case {case}")

    try:
        # Prepare inputs for Qwen2-VL
        print(f"Processing inputs for case {case}")
        text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        image_inputs, video_inputs = process_vision_info(messages)

        # Generate response with gradient calculations disabled
        with torch.no_grad():
            inputs = processor(
                text=[text],
                images=image_inputs,
                videos=video_inputs,
                padding=True,
                return_tensors="pt",
            ).to(model.device)

            generated_ids = model.generate(**inputs, max_new_tokens=4096, temperature=0.7, top_p=0.9)
            generated_ids_trimmed = [
                out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
            ]
            response_text = processor.batch_decode(
                generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
            )[0]


        print(f"Response generated for case {case}, length: {len(response_text)} characters")
        # Save the response
        response_path = os.path.join(case_dir, 'qwen-vl-2b-response.txt')
        with open(response_path, 'w', encoding='utf-8') as f:
            f.write(response_text)
        print(f"Response saved for case {case}.")
        # Memory management
        del inputs, generated_ids, response_text
        gc.collect()
        torch.cuda.empty_cache()
        print(f"Memory cleared for case {case}")
        

    except torch.cuda.OutOfMemoryError:
        print(f"CUDA out of memory error for case {case}. Skipping this case.")
        failed_cases.append(case)
        continue
    except KeyboardInterrupt:
        print("Interrupted by user. Proceeding to the next case.")
        continue
    except Exception as e:
        print(f"Error processing case {case}: {e}")
        failed_cases.append(case)
        continue

# After processing all cases, save failed cases
failed_cases_path = os.path.join(cases_dir, 'failed_cases_2.txt')
with open(failed_cases_path, 'w', encoding='utf-8') as f:
    for failed_case in failed_cases:
        f.write(f"{failed_case}\n")
print(f"Failed cases logged in {failed_cases_path}")

Found 9 image files for case RHUH-0019
Reading clinical information for case RHUH-0019
Loaded image: RHUH-0019_batch_1.png for case RHUH-0019
Loaded image: RHUH-0019_batch_2.png for case RHUH-0019
Loaded image: RHUH-0019_batch_3.png for case RHUH-0019
Loaded image: RHUH-0019_batch_4.png for case RHUH-0019
Loaded image: RHUH-0019_batch_5.png for case RHUH-0019
Loaded image: RHUH-0019_batch_6.png for case RHUH-0019
Loaded image: RHUH-0019_batch_7.png for case RHUH-0019
Loaded image: RHUH-0019_batch_8.png for case RHUH-0019
Loaded image: RHUH-0019_batch_9.png for case RHUH-0019
Messages prepared for case RHUH-0019
Processing inputs for case RHUH-0019
Response generated for case RHUH-0019, length: 2984 characters
Response saved for case RHUH-0019.
Memory cleared for case RHUH-0019
Found 6 image files for case RHUH-0001
Reading clinical information for case RHUH-0001
Loaded image: RHUH-0001_batch_1.png for case RHUH-0001
Loaded image: RHUH-0001_batch_2.png for case RHUH-0001
Loaded image: R

In [8]:
torch.cuda.empty_cache()
failed_cases_path = os.path.join(cases_dir, 'failed_cases.txt')
if not os.path.exists(failed_cases_path):
    print("No failed cases to reprocess.")
    exit()

with open(failed_cases_path, 'r', encoding='utf-8') as f:
    failed_cases = [line.strip() for line in f.readlines()]

# Initialize a new list for cases that fail again
new_failed_cases = []

# Set a maximum number of retries
max_retries = 2

for case in failed_cases:
    retry_count = 0
    while retry_count < max_retries:
        try:
            case_dir = os.path.join(cases_dir, case)
            response_path = os.path.join(case_dir, 'qwen-vl-72b-response.txt')
            if os.path.exists(response_path):
                print(f"Response already exists for case {case}. Skipping.")
                break  # Skip if response already exists

            # Load image and clinical information (same as before)
            image_files = [f for f in os.listdir(case_dir) if f.lower().endswith('.png')]
            clinical_information_path = os.path.join(case_dir, 'diagnostic_prompt.txt')
            clinical_information = open(clinical_information_path, 'r', encoding='utf-8').read()

            # Prepare messages (same as before)
            system_prompt = """Consider that you are a professional radiologist with several years of experience and you are now treating a patient. Write a fully detailed diagnosis report for this case, avoiding any potential hallucination and paying close attention to all of the batch images attached to this message.

Use the following structure for the report:

## Radiologist's Report

### Patient Information:
- *Age:* 65
- *Sex:* Male
- *Days from earliest imaging to surgery:* 1
- *Histopathological Subtype:* Glioblastoma
- *WHO Grade:* 4
- *IDH Status:* Mutant
- *Preoperative KPS:* 80
- *Preoperative Contrast-Enhancing Tumor Volume (cm³):* 103.21
- *Preoperative T2/FLAIR Abnormality (cm³):* 36.29
- *Extent of Resection (EOR %):* 100.0
- *EOR Type:* Gross Total Resection (GTR)
- *Adjuvant Therapy:* Radiotherapy (RT) + Temozolomide (TMZ)
- *Progression-Free Survival (PFS) Days:* 649
- *Overall Survival (OS) Days:* 736

#### Tumor Characteristics:

#### Segmentation Analysis:

#### Surgical Considerations:

### Clinical Summary:

### Recommendations:

### Prognostic Considerations:

### Follow-Up Plan:
 
### Additional Notes*(if any)*:

Ensure all findings from all of the images and clinical data provided. Please mention at the end of the report how many images were reviewed."""

            user_prompt = f"""You will be given batches of images, which are different sequences of MRI scans. 
    The images are for patients who are likely to have a brain tumor. Each image will contain up to 10 slices for 5 different sequences and the segmentation masks for the tumor at the bottom row of the image. 
    Additional clinical data about the patient is: 
    {clinical_information}"""

            # Collect the last image (same as before)
            last_image = None
            for image_file in image_files:
                image_path = os.path.join(case_dir, image_file)
                img = load_image(image_path)
                if img is not None:
                    last_image = {"type": "image", "image": img}

            if last_image is None:
                print(f"No valid images found for case: {case}")
                new_failed_cases.append(case)
                break  # Skip this case

            messages = [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": [last_image, {"type": "text", "text": user_prompt}]},
            ]

            # Prepare inputs and generate response (same as before)
            text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
            image_inputs, video_inputs = process_vision_info(messages)

            with torch.no_grad():
                inputs = processor(
                    text=[text],
                    images=image_inputs,
                    videos=video_inputs,
                    padding=True,
                    return_tensors="pt",
                ).to("cuda")

                generated_ids = model.generate(**inputs, max_new_tokens=4096, temperature=0.7, top_p=0.9)
                generated_ids_trimmed = [
                    out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
                ]
                response_text = processor.batch_decode(
                    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
                )[0]

            # Save the response
            with open(response_path, 'w', encoding='utf-8') as f:
                f.write(response_text)

            # Memory management
            del inputs, generated_ids, response_text
            gc.collect()
            torch.cuda.empty_cache()

            print(f"Response saved for case {case} after retry.")
            break  # Success, no need to retry

        except torch.cuda.OutOfMemoryError:
            print(f"CUDA out of memory error for case {case} on retry {retry_count + 1}. Retrying...")
            retry_count += 1
            continue
        except Exception as e:
            print(f"Error processing case {case} on retry {retry_count + 1}: {e}")
            retry_count += 1
            continue

    if retry_count == max_retries:
        print(f"Failed to process case {case} after {max_retries} retries. Adding to new failed cases.")
        new_failed_cases.append(case)

# Save new failed cases
new_failed_cases_path = os.path.join(cases_dir, '72_new_failed_cases.txt')
with open(new_failed_cases_path, 'w', encoding='utf-8') as f:
    for failed_case in new_failed_cases:
        f.write(f"{failed_case}\n")
print(f"New failed cases logged in {new_failed_cases_path}")

Response already exists for case RHUH-0001. Skipping.
Response already exists for case RHUH-0004. Skipping.
Response already exists for case RHUH-0007. Skipping.
Response already exists for case RHUH-0009. Skipping.
Response already exists for case RHUH-0014. Skipping.
Response already exists for case RHUH-0015. Skipping.
Response already exists for case RHUH-0018. Skipping.
Response already exists for case RHUH-0020. Skipping.
Response already exists for case RHUH-0022. Skipping.
Response already exists for case RHUH-0023. Skipping.
Response already exists for case RHUH-0025. Skipping.
Response already exists for case RHUH-0026. Skipping.
Response already exists for case RHUH-0027. Skipping.
Response already exists for case RHUH-0035. Skipping.
Response saved for case RHUH-0040 after retry.
New failed cases logged in /media/RLAB-Disk01/(final)merged_images_with_labels_order_and_folders_mask_normalized/72_new_failed_cases.txt
